[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jkitchin/s26-06642/blob/main/dsmles/12-uncertainty-quantification/uncertainty-quantification.ipynb)

In [ ]:
! pip install -q pycse
from pycse.colab import pdf

# Module 10: Uncertainty Quantification

Quantifying confidence in model predictions.

## Learning Objectives

1. Understand why uncertainty matters in engineering
2. Use pycse for regression with confidence intervals
3. Apply nonlinear fitting with uncertainty
4. Use Gaussian Processes for probabilistic predictions
5. Propagate uncertainty through calculations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

# pycse for uncertainty quantification
from pycse import regress, nlinfit

## Why Uncertainty Matters: The Engineering Imperative

In science, we publish point estimates. In engineering, we design with safety margins. The difference is **uncertainty**.

### Real-World Consequences

| Scenario | What You Predict | What You Need to Know |
|----------|------------------|----------------------|
| Reactor design | Optimal temperature: 450 K | Is it 450 ± 5 K or 450 ± 50 K? |
| Material strength | Mean strength: 100 MPa | What's the 1% failure threshold? |
| Process optimization | Expected yield: 85% | What's the range we'll actually see? |
| Economic analysis | Predicted cost: $1M | Could it be $2M? $500K? |

### Types of Uncertainty

1. **Measurement uncertainty**: Sensor noise, calibration errors
2. **Model uncertainty**: Model is an approximation of reality
3. **Parameter uncertainty**: Fitted parameters have error
4. **Extrapolation uncertainty**: Predictions outside training range

### The Honest Scientist's Burden

**A prediction without uncertainty is incomplete.** When you report "conversion = 75%," you're implicitly claiming infinite precision. Better: "conversion = 75 ± 5%"

This module teaches you how to quantify and propagate uncertainty—essential skills for engineering practice.

## Linear Regression with pycse: Parameters + Confidence

The `pycse` library (Python for Computational Science and Engineering) makes uncertainty quantification easy. The `regress` function returns not just fitted parameters, but their confidence intervals.

### Why This Matters

Standard scikit-learn gives you coefficients but not their uncertainties. For engineering decisions, you need to know:
- Is this coefficient significantly different from zero?
- How precise is our estimate?
- What's the range of predictions we should expect?

### The Output Format

`regress(X, y)` returns:
- `p`: Parameter estimates
- `pint`: 95% confidence intervals for each parameter
- `se`: Standard errors

In [ ]:
# Create experimental data: reaction rate vs temperature
np.random.seed(42)

# Arrhenius-like data (linearized): ln(k) = ln(A) - Ea/(R*T)
T = np.array([300, 320, 340, 360, 380, 400, 420, 440])  # Temperature (K)
R = 8.314  # Gas constant (J/mol/K)

# True parameters
Ea_true = 50000  # Activation energy (J/mol)
A_true = 1e8     # Pre-exponential factor

# Generate noisy data
ln_k_true = np.log(A_true) - Ea_true / (R * T)
ln_k = ln_k_true + np.random.normal(0, 0.3, len(T))

plt.figure(figsize=(10, 6))
plt.scatter(1/T * 1000, ln_k, s=80, label='Experimental data')
plt.xlabel('1000/T (1/K)')
plt.ylabel('ln(k)')
plt.title('Arrhenius Plot')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Linear regression with pycse.regress
# Model: ln(k) = b0 + b1 * (1/T)
# where b0 = ln(A) and b1 = -Ea/R

X = np.column_stack([np.ones(len(T)), 1/T])
y = ln_k

# regress returns: parameters, confidence intervals, predicted values
p, pint, se = regress(X, y, alpha=0.05)

print("Linear Regression Results (95% CI):")
print(f"  ln(A) = {p[0]:.3f} ± {(pint[0,1]-pint[0,0])/2:.3f}")
print(f"  -Ea/R = {p[1]:.1f} ± {(pint[1,1]-pint[1,0])/2:.1f}")

# Convert to physical parameters
A_est = np.exp(p[0])
Ea_est = -p[1] * R

print(f"\nPhysical Parameters:")
print(f"  A = {A_est:.2e} (true: {A_true:.2e})")
print(f"  Ea = {Ea_est/1000:.1f} kJ/mol (true: {Ea_true/1000:.1f} kJ/mol)")

**What pycse gives us that scikit-learn doesn't:**

The `regress` function returns **confidence intervals** on the parameters, not just point estimates. This is crucial for engineering:

- We know ln(A) = 18.5, but the 95% CI tells us it could reasonably be 18.2 to 18.8
- We know Ea/R ≈ 6000 K, but the CI shows our uncertainty

**Physical interpretation:**
- The estimated activation energy Ea ≈ 50 kJ/mol matches our "true" value
- The pre-exponential factor A ≈ 10⁸ s⁻¹ is also recovered well
- The narrow confidence intervals indicate precise parameter estimates (good data quality)

This is the difference between "our activation energy is 50 kJ/mol" (overconfident) and "our activation energy is 50 ± 3 kJ/mol at 95% confidence" (honest science).

In [ ]:
# Plot with confidence bands
T_plot = np.linspace(290, 450, 100)
X_plot = np.column_stack([np.ones(len(T_plot)), 1/T_plot])

# Predicted values
ln_k_pred = X_plot @ p

# Confidence interval for predictions
from scipy import stats
n = len(T)
dof = n - 2
t_val = stats.t.ppf(0.975, dof)
residuals = y - X @ p
mse = np.sum(residuals**2) / dof

# Standard error of prediction
XtX_inv = np.linalg.inv(X.T @ X)
se_pred = np.sqrt(mse * np.array([x @ XtX_inv @ x for x in X_plot]))
ci = t_val * se_pred

plt.figure(figsize=(10, 6))
plt.scatter(1/T * 1000, ln_k, s=80, label='Data', zorder=5)
plt.plot(1/T_plot * 1000, ln_k_pred, 'r-', linewidth=2, label='Fit')
plt.fill_between(1/T_plot * 1000, ln_k_pred - ci, ln_k_pred + ci, 
                 alpha=0.3, color='red', label='95% CI')
plt.xlabel('1000/T (1/K)')
plt.ylabel('ln(k)')
plt.title('Arrhenius Fit with Confidence Interval')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Nonlinear Fitting with pycse

The `pycse.nlinfit` function handles nonlinear models with uncertainty.

In [ ]:
# Michaelis-Menten kinetics: V = Vmax * S / (Km + S)
np.random.seed(42)

S = np.array([0.5, 1, 2, 4, 8, 16, 32, 64])  # Substrate concentration
Vmax_true = 100
Km_true = 5

V_true = Vmax_true * S / (Km_true + S)
V = V_true + np.random.normal(0, 3, len(S))

plt.figure(figsize=(10, 6))
plt.scatter(S, V, s=80, label='Data')
plt.xlabel('Substrate Concentration [S]')
plt.ylabel('Reaction Rate V')
plt.title('Michaelis-Menten Kinetics')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Define the model function
def michaelis_menten(S, Vmax, Km):
    return Vmax * S / (Km + S)

# Initial guesses
p0 = [80, 3]

# Nonlinear fit with pycse
p, pint, se = nlinfit(michaelis_menten, S, V, p0, alpha=0.05)

print("Nonlinear Regression Results (95% CI):")
print(f"  Vmax = {p[0]:.2f} ± {(pint[0,1]-pint[0,0])/2:.2f} (true: {Vmax_true})")
print(f"  Km = {p[1]:.2f} ± {(pint[1,1]-pint[1,0])/2:.2f} (true: {Km_true})")

In [ ]:
# Plot the fit
S_plot = np.linspace(0.1, 70, 200)
V_fit = michaelis_menten(S_plot, *p)

plt.figure(figsize=(10, 6))
plt.scatter(S, V, s=80, label='Data', zorder=5)
plt.plot(S_plot, V_fit, 'r-', linewidth=2, label='Fit')
plt.axhline(y=p[0], color='gray', linestyle='--', alpha=0.5, label=f'Vmax = {p[0]:.1f}')
plt.axvline(x=p[1], color='gray', linestyle=':', alpha=0.5, label=f'Km = {p[1]:.1f}')
plt.xlabel('Substrate Concentration [S]')
plt.ylabel('Reaction Rate V')
plt.title('Michaelis-Menten Fit')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Gaussian Process Regression: Uncertainty That Knows What It Doesn't Know

Gaussian Processes (GPs) are fundamentally different from other regression methods. They don't just give predictions—they give **probability distributions** over predictions.

### The Key Insight

A GP knows where it has data and where it doesn't. Predictions close to training points are confident (small uncertainty). Predictions far from training points are uncertain (large uncertainty).

This is exactly what we want for engineering! If you're extrapolating, the model should tell you it's uncertain.

### When to Use GPs

| Situation | GP Advantage |
|-----------|--------------|
| Expensive experiments | Guides where to sample next |
| Optimization | Balances exploration vs exploitation |
| Sparse data | Interpolates smoothly with uncertainty |
| Critical predictions | Honest about what it doesn't know |

### The Tradeoff

GPs are powerful but:
- Scale poorly to large datasets (O(n³) training)
- Require kernel selection (domain knowledge helps)
- Can underestimate uncertainty far from training data

In [ ]:
# Create training data
np.random.seed(42)

X_train = np.array([1, 2, 3, 5, 6, 8]).reshape(-1, 1)
y_train = np.sin(X_train).ravel() + np.random.normal(0, 0.1, len(X_train))

# Define kernel
kernel = ConstantKernel(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1)

# Fit GP
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp.fit(X_train, y_train)

print(f"Optimized kernel: {gp.kernel_}")

**Notice the key feature of Gaussian Processes:**

1. **Near training data (x=1-8)**: The uncertainty band is narrow. The GP is confident because it has data here.

2. **Far from training data (x=0-1, x=8-10)**: The uncertainty band widens dramatically. The GP is honest about what it doesn't know.

3. **Between training points**: Uncertainty is intermediate—reasonable interpolation.

**Compare to standard regression**: A linear or polynomial model would give the same confidence interval everywhere. It doesn't "know" that x=9 is far from training data while x=5 is close to training points.

**The GP's uncertainty is data-aware.** This is exactly what we want for engineering applications:
- Confident predictions where we have experimental support
- Honest uncertainty where we're extrapolating
- Guidance on where to collect more data (the uncertain regions!)

In [ ]:
# Predict with uncertainty
X_test = np.linspace(0, 10, 100).reshape(-1, 1)
y_pred, y_std = gp.predict(X_test, return_std=True)

plt.figure(figsize=(12, 6))

# Plot predictions with uncertainty
plt.plot(X_test, y_pred, 'b-', linewidth=2, label='GP Mean')
plt.fill_between(X_test.ravel(), 
                 y_pred - 1.96*y_std, 
                 y_pred + 1.96*y_std, 
                 alpha=0.3, color='blue', label='95% CI')
plt.fill_between(X_test.ravel(), 
                 y_pred - y_std, 
                 y_pred + y_std, 
                 alpha=0.3, color='blue')

# Plot training data
plt.scatter(X_train, y_train, c='red', s=100, zorder=5, label='Training data')

# Plot true function
plt.plot(X_test, np.sin(X_test), 'k--', alpha=0.5, label='True function')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Gaussian Process Regression with Uncertainty')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# GP for chemical engineering: Catalyst activity prediction
np.random.seed(42)

# Sparse training data
temp_train = np.array([300, 350, 400, 500, 550]).reshape(-1, 1)
activity_train = np.array([10, 35, 75, 95, 85])  # Activity peaks, then decreases (sintering)

# Fit GP
kernel = ConstantKernel(100) * RBF(length_scale=50) + WhiteKernel(noise_level=5)
gp_cat = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp_cat.fit(temp_train, activity_train)

# Predict
temp_test = np.linspace(280, 600, 100).reshape(-1, 1)
activity_pred, activity_std = gp_cat.predict(temp_test, return_std=True)

plt.figure(figsize=(12, 6))
plt.plot(temp_test, activity_pred, 'b-', linewidth=2, label='GP Prediction')
plt.fill_between(temp_test.ravel(), 
                 activity_pred - 1.96*activity_std, 
                 activity_pred + 1.96*activity_std, 
                 alpha=0.3, color='blue', label='95% CI')
plt.scatter(temp_train, activity_train, c='red', s=100, zorder=5, label='Experiments')

plt.xlabel('Temperature (K)')
plt.ylabel('Catalyst Activity')
plt.title('Catalyst Activity Prediction with Uncertainty')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Find optimal temperature
opt_idx = np.argmax(activity_pred)
print(f"\nOptimal temperature: {temp_test[opt_idx, 0]:.0f} K")
print(f"Expected activity: {activity_pred[opt_idx]:.1f} ± {1.96*activity_std[opt_idx]:.1f}")

## Uncertainty Propagation: How Errors Compound

When inputs have uncertainty, outputs have uncertainty too. But how much?

### Two Approaches

| Method | How It Works | When to Use |
|--------|--------------|-------------|
| **Analytical** | Taylor expansion, linear approximation | Simple functions, small uncertainties |
| **Monte Carlo** | Sample inputs, compute outputs, analyze distribution | Complex functions, any uncertainty size |

### The uncertainties Package

For analytical propagation, the `uncertainties` package is elegant. Define uncertain numbers, and it automatically tracks how uncertainty flows through calculations.

```python
T = ufloat(400, 5)  # 400 ± 5
P = ufloat(10, 1)   # 10 ± 1
result = P / T      # Automatically tracks uncertainty!
```

### When Monte Carlo Is Necessary

Analytical propagation assumes small, symmetric uncertainties. Use Monte Carlo when:
- Uncertainties are large
- Distributions are non-normal (e.g., uniform, log-normal)
- Functions are highly nonlinear
- You need the full output distribution, not just mean ± std

In [ ]:
# Using the uncertainties package (commonly used with pycse)
from uncertainties import ufloat
from uncertainties import umath

# Define uncertain quantities
T = ufloat(400, 5)     # Temperature: 400 ± 5 K
P = ufloat(10, 0.5)    # Pressure: 10 ± 0.5 bar
R = 8.314              # Gas constant (exact)

# Ideal gas: n/V = P/(RT)
# Convert P to Pa: P * 1e5
concentration = (P * 1e5) / (R * T)

print("Uncertainty Propagation Example:")
print(f"  Temperature: {T}")
print(f"  Pressure: {P} bar")
print(f"  Concentration: {concentration:.2f} mol/m³")

In [ ]:
# Arrhenius equation with uncertain parameters
A = ufloat(1.2e8, 0.3e8)       # Pre-exponential factor
Ea = ufloat(52000, 2000)       # Activation energy (J/mol)
T = ufloat(450, 10)            # Temperature (K)
R = 8.314

# k = A * exp(-Ea/(R*T))
k = A * umath.exp(-Ea / (R * T))

print("Arrhenius Rate Constant:")
print(f"  A = {A}")
print(f"  Ea = {Ea} J/mol")
print(f"  T = {T} K")
print(f"  k = {k:.2e}")

In [ ]:
# Monte Carlo uncertainty propagation
np.random.seed(42)
n_samples = 10000

# Sample from distributions
A_samples = np.random.normal(1.2e8, 0.3e8, n_samples)
Ea_samples = np.random.normal(52000, 2000, n_samples)
T_samples = np.random.normal(450, 10, n_samples)

# Calculate k for each sample
k_samples = A_samples * np.exp(-Ea_samples / (8.314 * T_samples))

plt.figure(figsize=(10, 6))
plt.hist(k_samples, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.axvline(x=np.mean(k_samples), color='r', linestyle='--', linewidth=2, label=f'Mean: {np.mean(k_samples):.2e}')
plt.axvline(x=np.percentile(k_samples, 2.5), color='g', linestyle=':', linewidth=2)
plt.axvline(x=np.percentile(k_samples, 97.5), color='g', linestyle=':', linewidth=2, label='95% CI')
plt.xlabel('Rate Constant k')
plt.ylabel('Probability Density')
plt.title('Monte Carlo Uncertainty Propagation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Monte Carlo Results:")
print(f"  Mean k: {np.mean(k_samples):.3e}")
print(f"  Std k: {np.std(k_samples):.3e}")
print(f"  95% CI: [{np.percentile(k_samples, 2.5):.3e}, {np.percentile(k_samples, 97.5):.3e}]")

## Summary: Uncertainty Quantification Toolkit

### When to Use What

| Situation | Recommended Method |
|-----------|-------------------|
| Linear regression parameters | `pycse.regress` |
| Nonlinear fitting parameters | `pycse.nlinfit` |
| Predictions with honest uncertainty | Gaussian Processes |
| Simple error propagation | `uncertainties` package |
| Complex or large uncertainties | Monte Carlo simulation |

### Key Takeaways

1. **Always report uncertainty**: A prediction without confidence limits is incomplete
2. **Confidence intervals ≠ prediction intervals**: Parameter uncertainty ≠ outcome uncertainty
3. **GPs grow uncertain far from data**: This is a feature, not a bug
4. **Propagate uncertainty through calculations**: Inputs uncertain → outputs uncertain
5. **Use Monte Carlo for complex cases**: When in doubt, sample

### The Engineering Mindset

In research, we seek the "true" value. In engineering, we design for the worst case. Understanding uncertainty lets you:
- Set appropriate safety factors
- Make robust decisions
- Plan experiments efficiently
- Communicate confidence honestly

### Common Pitfalls

- Ignoring model uncertainty (only reporting parameter uncertainty)
- Assuming normal distributions when data suggests otherwise
- Over-interpreting GP uncertainty far from training data
- Forgetting to propagate uncertainty to final decisions

## Next Steps

In the final module, we'll learn about model interpretability—understanding *why* models make their predictions, not just what they predict.

---

## The Catalyst Crisis: Chapter 12 - "What We Don't Know"

*A story about uncertainty, honesty, and building trust*

---

"You're recommending we reject 40% of incoming catalyst lots."

The ChemCorp VP of Operations had joined the call—someone none of them had met before. Senior enough to make decisions. Skeptical enough to question everything.

"Based on the clustering model, yes," Alex said.

"That's a $2 million annual cost. You're sure?"

The question hung in the air. Was she sure? Her model had 0.92 R-squared. Her clusters were statistically significant. The evidence pointed clearly at the catalyst.

But *sure*?

"I'd like to show you the uncertainty in our predictions," Alex said.

She pulled up the Gaussian Process model she'd built as a complement to the Random Forest. Unlike the forest, the GP provided confidence intervals—not just predictions, but estimates of how confident those predictions were.

"For batches using Cluster 1 catalyst, our predictions are tight. 95% confidence interval of plus or minus 3% yield." She clicked to the next plot. "For Cluster 3 catalyst, the confidence interval is plus or minus 12%."

The VP frowned. "So you're less certain about the bad catalyst?"

"We have less data from those conditions. The model knows what it doesn't know." Alex highlighted the uncertainty bands. "If you want to reduce this uncertainty, you could run controlled experiments—deliberately use some Cluster 3 catalyst under carefully monitored conditions."

"You're suggesting we intentionally run bad batches?"

"I'm suggesting you could learn more with a few planned experiments than with months of observational data. Right now, we're confident that Cluster 3 is worse. We're less confident about exactly how much worse, or whether operating conditions could compensate."

The room was quiet. This wasn't the clear answer they wanted. But it was the honest answer.

The VP surprised her. "I appreciate that. Too many consultants pretend to certainty they don't have." He leaned back. "We'll start with screening—reject the obvious Cluster 3 lots. And we'll design an experiment for the borderline cases. Your team will help?"

"Absolutely."

After the call, Jordan found Alex at her desk, staring at the uncertainty plots.

"That was brave. Telling an executive you're not sure."

Alex shrugged. "I was sure about what I was sure about. And honest about what I wasn't. That's all we can do."

She added to the mystery board: **Uncertainty is information. High confidence in Cluster 1 recommendations. Lower confidence in Cluster 3—need controlled experiments.**

---

*Continue to the final lecture for the resolution of the Catalyst Crisis...*